# Модель памяти в С++

## Стек

Чтобы оценивать эффективность кода, нужно понимать принципы его оптимазации, а также
сложность алгоритмов. Для этого нужно иметь представление о том, как хранятся объекты,
создаваемые в процессе работы программ, и как программы на
C++ взаимодействуют с оперативной памятью компьютера.

<br>
<br>

Рассмотрим сразу же **пример**.<br>
Допустим, перед нами стоит задача написать программу, которая будет бросать «флешку на А» <br>
(прим. [экскурс в мем, 2020г](https://vk.com/videos-58771215?z=video-58771215_456239189%2Fclub58771215%2Fpl)).<br>

Есть 3 функции, у каждой из которых есть локальные переменные.<br>
Вопрос поставлен следующим образом: **где в памяти хранятся все эти локальные переменные ?**

In [ ]:
#include <iostream>

using namespace std;

void na_a() {
    int i2 = 1;
    float f2 = 2.2; 
}

void fleshka() () {
    int i1 = 3;
    char c1 = '4';
    na_a();
}

int main() {
    int m = 5;
    char c = '6';
    fleshka();
    na_a();
    m = 7;
    c = '8';
}

Начнём разбираться. <br>
Все переменные хранятся в специальной области памяти, которая называется **стек**. <br>


Когда в программе запускается очередная функция, то на стеке
резервируется блок памяти, достаточный для хранения ее локальных переменных. Кроме того, в этом блоке хранится служебная информация — такая,
как, например, адрес возврата. Этот блок памяти называется **стековым фреймом функции**. Большой смысл в это понятие вкладывать не нужно — это просто область памяти (рамка), которая выделяется всякий раз, когда вызывается функция, и которая хранит временные параметры.

<img src="photos/2.png">

__Что происходит?__

Функция <code>main()</code> запускает функцию <code>fleshka()</code> и на стеке ниже фрейма функции <code>main()</code> резервируется фрейм для функции <code>fleshka()</code> (см.ниже). То же самое с функцией <code>na_a()</code>, которая запускается в функции <code>fleshka()</code>.


[Про понятие стека вызова](https://foxford.ru/wiki/informatika/stek-vyzovov)

<img src="photos/3.png" width="200">

Когда функция завершает свою работу,
то вершина стека просто перемещается на фрейм предыдущей функции.
На картинке ниже вызов функции <code>na_a()</code> из функции <code>fleshka()</code> завершился. <br>
(фрейм функции <code>na_a()</code> стал бледным, т.е. вершина стека
поднялась до фрейма функции <code>fleshka()</code>)<br>

Важно! Сам фрейм функции <code>na_a()</code> остался на стеке! То есть, данные, которые создала эта функция остались - ничего не чистилось. Происходит просто перемещение вершины стека.

<img src="photos/4.png" width="200">

<code>fleshka()</code>закончила свю работу и вершина стека переместилась на функцию <code>main()</code>.

<img src="photos/5.png" width="200">

Вызываем функцию <code>na_a()</code> из функции <code>main()</code>. Стековый фрейм функции <code>na_a()</code> перетирает данные, которые были в стеке от предыдущих вызовов.

P.S. Неровной таблицей я показал, что данные остались от предыдущих вызовов и новые на них «ложатся».

<img src="photos/7.png" height="50" width="200">

Дальше мы выходим из функцию <code>na_a()</code> и <code>main()</code>. Вершина стека поднялась до самого верха.

Исходя из этой структуры, становится понятно, почему значение непроинициализированной переменной не определено.



***

## Куча

Проблема:

В ходе работы программы вызываются разные функции и, соответственно,
память на стеке перезатирается от других вызовов функций. Для хранения данных стек не подходит (пример - вектор, который должен хранить свои элементы на протяжении всей его «жизни». Умные люди сделали вывод: нужно найти другой источник памяти. <br> <br>
Решение - **heap** **(куча)**.

Программа может
в любой момент обратиться в кучу и попросить у неё об одолжении - «дай мне, пожалуйста, немного памяти произвольного размера в долг». Куча выделяет блок памяти и взамен требует, чтобы программа вернула (освободила) её же ресурс.

Отличие от стека состоит в том, что здесь каждый вызов сопровождается созданием отдельного блока памяти, достаточного для хранения элементов.

Микровывод: если нам нужно, чтобы наши данные жили дольше, чем создавшая их функция, нужно использовать кучу. Наша обычная ситуация: когда мы не знаем, сколько элементов должно находиться в массиве, мы обращаемся в кучу за временной памятью.


***

## Практика

Как работать со стеком, думаю, более-менее понятно всем.

Работа с кучей не кажется такой тривиальной. Посмотрим несколько **примеров** $+$ затронем указатели.

**Пример 1.**

Выделить в куче память для хранения одного значения типа int.

In [ ]:
int* p = new int;

__Что происходит?__


1. $p$ - локальная переменная $\Rightarrow$ будет храниться на стеке.
2. тип переменной $p - int*$.
3. инициализируем переменную $p$ значением $new$  $int$
4. $new$ - оператор, который выделяет память из кучи.
5. $int$ - подсказка оператору $new$ значение какого типа будем хранить.
6. $p$ - указатель - адрес в памяти (индекс в памяти). Память -  линейный массив байт $\Rightarrow$ указатель - это индекс в массиве байт.
<img src="photos/8.png" height="50" width="250">
7. как обратиться к значению, на которое указатель «указывает» ?
   синтаксис такой: 
   
   $*p = [значение]$.
8. чтобы опеределить значение при инициализации нужно написать так:

    $int* p = new$  $int([значение])$.

**Пример 2.**

In [7]:
{
    int* p5 = new int(5);
}

<img src="photos/9.png" width="250">

__Что происходит?__

Происходит беда. После выхода из области видимости указатель p5 умрёт (см.ниже), а $5$ останется в куче. Память выделилась, но её никто не освободил. Указатель на память потеряли $\Rightarrow$ связь с этой $5$ мы потеряли.

<img src="photos/10.png" width="250">

Такая уж ли эта беда?

Пример игрушечный, потеряли пятёрку - ну ничего, бывает, процесс закроет глаза на эту мелкую утечку и закончит свою работу.

Однако если мы работаем с большими данными, то утечка может привести к падению программы и приложения в целом, если речь идёт о серьезных многофайловых проектах. Также эта проблема опасна для процессов, когда важна бесперебойная работа.

In [ ]:
delete p;

Как вы помните, мы в долгу перед кучей - нужно освободить ей память. Оператор $delete$ это и делает.

***
***
***